In [1]:
import requests
import time
import os
import sys
import re
import io
import json
import csv
import openpyxl
import pandas as pd
import numpy as np
from urllib.request import urlopen
from urllib.error import URLError, HTTPError
from bs4 import BeautifulSoup
from selenium.webdriver import Chrome
from selenium.webdriver.common.keys import Keys
from konlpy.tag import Komoran
from nltk import Text
from pykospacing import spacing
import kss

In [3]:
browser = Chrome()

titles = []
links = []
    
#링크, 제목
for i in range(1,3):
    browser.get("https://pann.nate.com/talk/ranking?rankingType=life&page=" + str(i))
    soup = BeautifulSoup(browser.page_source, 'html.parser')

    first_list = soup.find('div', {'class': 'cntList'}).findAll('li')

    for li in first_list: 
        f_link = li.findAll('a')
        for a in f_link:
            real_link = 'https://pann.nate.com' + a.get('href') 
        links.append(real_link)
    
    for li in first_list:
        f_title = li.findAll('dl')
        for dl in f_title:
            t = dl.find('a')
            real_title = t.get('title')
            real_title = spacing(real_title)
            real_title = re.sub('[ㄱ-ㅎㅏ-ㅣa-zA-Z0-9]', '', real_title)
            real_title = re.sub('[\xa0|0xed]', '', real_title)
            real_title = re.sub('[-=+_★♥♡,#/\?:╋^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》—;]','',real_title)
        titles.append(real_title)

#print({'제목': titles, 'url': links}, sep='\n')

In [4]:
#본문
txt = []
    
for i in links:
    try: 
        res = requests.get(i)
        res.raise_for_status()
        res.encoding = None            
        html2 = res.text
        
        soup = BeautifulSoup(html2, 'html.parser')
        contentArea = soup.find("div", {"class" : "viewarea"})            
        parags = contentArea.findAll("div", {"id" : "contentArea"})

        content = ""

        for parag in parags:
            content += parag.text
        content = re.sub('[ㄱ-ㅎㅏ-ㅣa-zA-Z0-9]', '', content)
        content = re.sub('[&nbsp;|\n|\t|\r]', '', content)
        content = re.sub('[\xa0]', '', content)
        content = re.sub('[-=+_★♥♡,#/\?:╋^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》—;]', '', content)
        content = spacing(content)
        
        txt.append(content)
             
    except HTTPError as e:
        txt.append('')
    except URLError as e:
        txt.append('')
    except AttributeError as e:
        txt.append('')
        
#print(txt)

In [5]:
#댓글
comments =[]

for i in links:
    try: 
        res = requests.get(i)
        res.raise_for_status()
        res.encoding = None            
        html2 = res.text
        
        soup = BeautifulSoup(html2, 'html.parser')
        contentArea = soup.find("div", {"class" : "cmt_list"})            
        parags = contentArea.findAll("dd", {"class" : "usertxt"})
        
        content = ""

        for parag in parags:
            content += parag.text
        content = re.sub('[ㄱ-ㅎㅏ-ㅣa-zA-Z0-9]', '', content)
        content = re.sub('[&nbsp;|\n|\t|\r]', '', content)
        content = re.sub('[\xa0]', '', content)
        content = re.sub('[-=+_★♥♡,#/\?:╋^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》—;]', '', content)    
        content = spacing(content)
        
        comments.append(content)
    
    except HTTPError as e:
        txt.append('')
    except URLError as e:
        txt.append('')
    except AttributeError as e:
        txt.append('')

#print(comments)
browser.close()

In [33]:
keyword = pd.read_csv('10키워드s.csv', encoding='utf-8')
keyword.values
klist = []

for i in keyword.values:
    klist.append(str(i))

#for문 추가 - 10개 loop
klist

["['갈등']",
 "['결혼']",
 "['남녀']",
 "['남자']",
 "['남편']",
 "['부모']",
 "['여자']",
 "['유발']",
 "['자기']",
 "['친구']"]

In [44]:
#형태소 분석
komoran = Komoran() 

title_morphs = []
txt_morphs = []
comment_morphs = []


for i in titles:
    title_morphs.append(komoran.morphs(i))
    
for i in txt:
    txt_morphs.append(komoran.morphs(i))
    
for i in comments:
    comment_morphs.append(komoran.morphs(i))  
        
nate_dict = {
    '제목' : titles,
    '본문' : txt,
    '댓글' : comments
}

df = pd.DataFrame(nate_dict) 
df.to_csv('natepann.csv', index=False, encoding="utf-8-sig")
    
morphs_dict = {
    '제목 형태소' : title_morphs,
    '본문 형태소' : txt_morphs,
    '댓글 형태소' : comment_morphs
}

df2 = pd.DataFrame(morphs_dict)
df2.to_csv('natepann_Morphs.csv', index=False, encoding="utf-8-sig")

In [46]:
# #형태소 분석
# komoran = Komoran() 

# title_morphs = []
# txt_morphs = []
# comment_morphs = []

# key = '갈등'

# for i in titles:
#     if key in i:
#         title_morphs.append(komoran.morphs(i))
    
# for i in txt:
#     if key in i:
#         txt_morphs.append(komoran.morphs(i))
    
# for i in comments:
#     if key in i:
#         comment_morphs.append(komoran.morphs(i))  
        
# morphs_dict = {
#     '제목 형태소' : title_morphs,
#     '본문 형태소' : txt_morphs,
#     '댓글 형태소' : comment_morphs
# }

# df = pd.DataFrame(nate_dict) 
# df.to_csv('keyword#1.csv', index=False, encoding="utf-8-sig")